Importing libraries and tools

In [92]:
import torch
import torchvision
import torchvision.transforms as transf
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


Setting up some basic parameters and checking the system available

In [93]:
epochs = 6
batchsize = 50
learnrate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Transforms for the dataset, including resizeing and the use of data augmentations

In [94]:
trans_train = transf.Compose([
    transf.Resize(size=(224, 224)),
    transf.RandomHorizontalFlip(),
    transf.RandomRotation(10),
    transf.ToTensor(),
    transf.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2)),])
trans_test = transf.Compose([
    transf.Resize(size=(224, 224)),
    transf.ToTensor(),
    transf.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2)),])

Downloading and Loading the data in 2 sets - Training data and Testing Data

In [95]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download= True, transform = trans_train)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download= True, transform = trans_test)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batchsize, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size = batchsize, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


The 10 classes in the dataset

In [96]:
classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

Importing and setting up the VGG16 model for Transfer Learning and altering the number of output features

In [97]:
from torchvision.models import vgg16, VGG16_Weights
weights = VGG16_Weights.DEFAULT
model = vgg16(weights=weights)
inputlastlayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(inputlastlayer,10)
model = model.to(device)


The loss function and optimiser

In [98]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr = learnrate, momentum = 0.9,weight_decay=5e-4)
total_step = len(train_dataloader)

Training the Model in steps according to the batchsize of the training and testing dataset, over a for loop

In [82]:
for epoch in range(epochs):
   for i, (image , labels) in enumerate(train_dataloader):
          image = image.to(device)
          labels = labels.to(device)

          labels_hat = model(image)
          corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
          loss_value = loss_func(labels_hat, labels)
          loss_value.backward()
          optimiser.step()
          optimiser.zero_grad()
          if (i+1) % 250 == 0:
            print(f'epoch {epoch+1}/{epochs}, step: {i+1}/{total_step}: loss = {loss_value:.5f}, acc = {100*(corrects/labels.size(0)):.2f}%')


epoch 1/6, step: 250/1000: loss = 0.48554, acc = 84.00%
epoch 1/6, step: 500/1000: loss = 0.30370, acc = 90.00%
epoch 1/6, step: 750/1000: loss = 0.40027, acc = 88.00%
epoch 1/6, step: 1000/1000: loss = 0.32233, acc = 86.00%
epoch 2/6, step: 250/1000: loss = 0.31275, acc = 86.00%
epoch 2/6, step: 500/1000: loss = 0.22429, acc = 92.00%
epoch 2/6, step: 750/1000: loss = 0.17925, acc = 94.00%
epoch 2/6, step: 1000/1000: loss = 0.42904, acc = 84.00%
epoch 3/6, step: 250/1000: loss = 0.09046, acc = 96.00%
epoch 3/6, step: 500/1000: loss = 0.24937, acc = 90.00%
epoch 3/6, step: 750/1000: loss = 0.26933, acc = 94.00%
epoch 3/6, step: 1000/1000: loss = 0.30932, acc = 90.00%
epoch 4/6, step: 250/1000: loss = 0.14717, acc = 92.00%
epoch 4/6, step: 500/1000: loss = 0.11802, acc = 96.00%
epoch 4/6, step: 750/1000: loss = 0.20078, acc = 92.00%
epoch 4/6, step: 1000/1000: loss = 0.09754, acc = 100.00%
epoch 5/6, step: 250/1000: loss = 0.09863, acc = 94.00%
epoch 5/6, step: 500/1000: loss = 0.14857, 

In [105]:
accurate = 0
total = 0

with torch.no_grad():
  for i, (testimages,testlabels) in enumerate(test_dataloader):
    testimages = testimages.to(device)
    testlabels = testlabels.to(device)
    predicted = model(testimages)
    labelspredicted = predicted.argmax(axis=1)
    accurate += (labelspredicted==testlabels).sum().item()
    total += testlabels.size(0)

print(f'Accuracy of the VGG16 CNN on the 10000 test images is : {100 * accurate / total} %')

Accuracy of the VGG16 CNN on the 10000 test images is : 93.48 %
